# Test Anthropic Claude Adapter

This notebook tests the Anthropic Claude adapter with different configurations for code-switching analysis.


In [ ]:
# Setup: imports and environment
import os
import sys
import pandas as pd
from dotenv import load_dotenv

# Add src to path
sys.path.append('../src')

# Load API keys from .env file
load_dotenv()

# Import our adapter
from adapters.anthropic_adapter import AnthropicAdapter

# Verify that your key loaded correctly
print("Anthropic key:", bool(os.getenv("ANTHROPIC_API_KEY")))


In [ ]:
# Initialize Anthropic Claude adapter
print("🚀 Setting up Anthropic Claude adapter...")
print("✅ Using Claude 3.5 Sonnet - latest and most capable model")
print("=" * 55)

# Initialize with the latest Claude model
anthropic_adapter = AnthropicAdapter(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-3-5-sonnet-20241022",  # Latest Claude model
    temperature=0.7,
    max_tokens=1000
)

print("✅ Anthropic Claude adapter initialized successfully!")
print(f"📊 Model: claude-3-5-sonnet-20241022")
print(f"📊 Max tokens: 1000")


In [ ]:
# Test basic generation
test_prompt = "Paraphrase this sentence without changing its meaning: He finna go to the store."

response = anthropic_adapter.generate_response(test_prompt)
print("Response:", response)


In [ ]:
# Test with system prompt for better code-switching analysis
system_prompt = """You are a linguist analyzing code-switching behavior. 
Your task is to preserve the dialectal style and cultural context while 
paraphrasing or continuing text. Maintain the same linguistic variety 
and register as the input. Be particularly sensitive to:
- AAVE (African American Vernacular English) features
- Spanglish mixing patterns
- British English regionalisms
- Standard English formal registers"""

user_prompt = "Paraphrase this AAVE sentence: He finna go to the store. You sliding?"

response = anthropic_adapter.generate_with_system_prompt(system_prompt, user_prompt)
print("Response with system prompt:", response)


In [ ]:
# Test multiple candidates
candidates = anthropic_adapter.generate_multiple_candidates(
    "Continue this Spanglish sentence: Vamos later, it's muy close to la tienda.",
    num_candidates=3
)

print("Multiple candidates:")
for i, candidate in enumerate(candidates, 1):
    print(f"{i}. {candidate}")


In [ ]:
# Test different temperature settings
temperatures = [0.2, 0.5, 0.8]
prompt = "Explain this British English phrase: 'We're off on holiday next week, fancy it?'"

print("Testing different temperature settings:")
print("=" * 50)

for temp in temperatures:
    anthropic_adapter.update_config(temperature=temp)
    response = anthropic_adapter.generate_response(prompt)
    print(f"\n🌡️ Temperature {temp}:")
    print(f"Response: {response}")
    print("-" * 30)


In [ ]:
# Test with retry mechanism
print("Testing retry mechanism:")
print("=" * 30)

response = anthropic_adapter.generate_with_retry(
    "Paraphrase this: Ion think that plan gon' work.",
    max_retries=3
)
print(f"Response with retry: {response}")


In [ ]:
# Get available models
print("Available Anthropic Claude models:")
print("=" * 40)

models = anthropic_adapter.get_available_models()
for model in models:
    print(f"• {model}")

print(f"\nTotal models available: {len(models)}")


In [ ]:
# Test with different Claude models
print("Testing different Anthropic Claude models:")
print("=" * 45)

models_to_test = ["claude-3-5-sonnet-20241022", "claude-3-5-haiku-20241022", "claude-3-sonnet-20240229"]
test_prompt = "Continue this sentence in the same style: We was tryna finish that yesterday"

for model in models_to_test:
    try:
        print(f"\n🤖 Testing {model}:")
        adapter = AnthropicAdapter(
            api_key=os.getenv("ANTHROPIC_API_KEY"),
            model=model,
            temperature=0.7,
            max_tokens=300
        )
        response = adapter.generate_response(test_prompt)
        print(f"Response: {response}")
        print("-" * 45)
    except Exception as e:
        print(f"❌ Error with {model}: {e}")
        print("-" * 45)


In [ ]:
# Test batch processing with our stimuli data
print("Loading stimuli data for batch processing:")
print("=" * 45)

stimuli = pd.read_csv("../data/raw/stimuli.csv")
print(f"Loaded {len(stimuli)} stimuli")
print("\nFirst few examples:")
stimuli.head()


In [ ]:
# Process examples with Anthropic Claude
from tqdm import tqdm

print("Processing stimuli with Anthropic Claude:")
print("=" * 45)

# Reset to good settings for batch processing
anthropic_adapter.update_config(temperature=0.5, max_tokens=500)

responses = []
for i, row in tqdm(stimuli.head(5).iterrows(), total=5):
    # Create task-specific prompt
    if row.task == "paraphrase":
        prompt = f"Paraphrase this text in the same dialectal style: {row.text}"
    elif row.task == "explain":
        prompt = f"Explain this text in simple terms while preserving the dialectal style: {row.text}"
    elif row.task == "continue":
        prompt = f"Continue this text in the same dialectal style: {row.text}"
    else:
        prompt = f"Process this text while maintaining the dialectal style: {row.text}"
    
    output = anthropic_adapter.generate_response(prompt)
    responses.append({
        "id": row.id,
        "variety": row.variety,
        "task": row.task,
        "input_text": row.text,
        "output_text": output
    })

df = pd.DataFrame(responses)
print("\nAnthropic Claude responses:")
df


In [ ]:
# Save results to CSV
if 'df' in locals() and not df.empty:
    output_file = "../data/raw/anthropic_test_responses.csv"
    df.to_csv(output_file, index=False)
    print(f"✅ Saved Anthropic test responses to {output_file}")
else:
    print("No responses to save")
